# May 24, 2025: subdivide the rois derived from allen annotations
Francesca shared Allen 120 rois parcellation. these rois are pretty big. subdivide each roi into multiple parts.

In [1]:
import csv
import os
import numpy as np
import pandas as pd
import dill as pickle
import nibabel as nib
from sklearn.cluster import KMeans

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100
np.random.seed(args.SEED)

In [3]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.makedirs(PARCELS_path, exist_ok=True)

In [4]:
args.source = 'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 120
args.resolution = 200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-120_res-200'

In [5]:
TEMP_DESC = (
    f'source-allen' #f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
TEMP_DESC

'source-allen_space-ccfv2_res-200'

In [6]:
template_file = f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'
template_img = nib.load(template_file)

In [7]:
parcels_file = f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
parcels_img = nib.load(parcels_file)
parcels_vol = np.round(parcels_img.get_fdata(), decimals=0).astype(int)
roi_table = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')
roi_table

,old_roi,name,roi
0,1,"R-Frontal pole, cerebral cortex (FRP,184)",1
1,2,"R-Somatomotor areas (MO,500)",2
2,3,"R-Somatosensory areas (SS,453)",3
3,4,"R-Gustatory areas (GU,1057)",4
4,5,"R-Visceral area (VISC,677)",5
...,...,...,...
115,156,"L-Medulla, sensory related (MY-sen,386)",116
116,157,"L-Medulla, motor related (MY-mot,370)",117
117,158,"L-Medulla, behavioral state related (MY-sat,379)",118
118,159,"L-Cerebellar cortex (CBX,528)",119


In [8]:
args.roi_size = 260 #150 #200 #250 #255 #260

In [9]:
def do_kmeans(args, div_vol):
    nonzero_voxels = list(zip(*np.where(div_vol)))
    nonzero_voxels = np.reshape(nonzero_voxels, (-1, len(div_vol.shape))).astype(int)

    num_rois = np.round(len(nonzero_voxels) / args.roi_size, decimals=0).astype(int)
    num_rois = num_rois if num_rois > 0 else 1
    # print(f'num_voxels:{len(nonzero_voxels)}, num_rois:{num_rois}')

    kmeans = KMeans(
        n_clusters=num_rois,
        init='k-means++',
        random_state=args.SEED,
    ).fit(nonzero_voxels.astype(float))
    rois = kmeans.predict(nonzero_voxels.astype(float)) + 1

    parcels = np.zeros_like(div_vol, dtype=int)
    for roi in np.unique(rois):
        vxls = nonzero_voxels[rois == roi]
        parcels[tuple(zip(*vxls))] = roi
        
    assert num_rois == len(np.unique(rois))
    return parcels, num_rois

In [10]:
new_rois = []
new_roi_names = []
new_parcels = np.zeros_like(parcels_vol, dtype=int)
for idx_row, row in roi_table.iterrows():
    # division: div
    div = row['roi']
    div_name = row['name']
    div_vol = (parcels_vol == div).astype(int)
    
    # if not div == 2: continue
    
    new_num_rois = len(np.unique(new_parcels)[1:])
    # print(div_name)
    parcels, num_rois = do_kmeans(args, div_vol)
    new_parcels += parcels + div_vol * new_num_rois
    
    roi_names = [
        f'{div_name} {i}'
        for i in range(1, num_rois+1)
    ]
    rois = range(new_num_rois+1, new_num_rois+num_rois+1)
    new_roi_names += roi_names
    new_rois += rois
    # break

new_num_rois = len(np.unique(new_parcels)[1:])
new_rois_table = pd.DataFrame(dict(
    old_roi=new_rois,
    name=new_roi_names,
    roi=new_rois,
))
new_rois_table
    

,old_roi,name,roi
0,1,"R-Frontal pole, cerebral cortex (FRP,184) 1",1
1,2,"R-Somatomotor areas (MO,500) 1",2
2,3,"R-Somatomotor areas (MO,500) 2",3
3,4,"R-Somatomotor areas (MO,500) 3",4
4,5,"R-Somatomotor areas (MO,500) 4",5
...,...,...,...
237,238,"L-Cerebellar cortex (CBX,528) 7",238
238,239,"L-Cerebellar cortex (CBX,528) 8",239
239,240,"L-Cerebellar cortex (CBX,528) 9",240
240,241,"L-Cerebellar cortex (CBX,528) 10",241


In [11]:
np.unique(new_parcels)[1:]

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [12]:
NEW_PARC_DESC = (
    f'source-allspl' #f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{new_num_rois}'
    f'_res-{args.resolution}'
)
NEW_PARC_DESC

'source-allspl_space-ccfv2_braindiv-whl_nrois-242_res-200'

In [13]:
new_parcels_img = nib.Nifti1Image(
    new_parcels,
    affine=parcels_img.affine,
    header=parcels_img.header,
)
nib.save(
    new_parcels_img, 
    f'{PARCELS_path}/{NEW_PARC_DESC}_desc-parcels.nii.gz'
)

In [14]:
new_rois_table.to_csv(
    f'{PARCELS_path}/{NEW_PARC_DESC}_desc-names.csv', 
    index=False
)